In [5]:
from bs4 import BeautifulSoup
import urllib.request as req
import pandas as pd
import os
label_path = "./dataset/MovieGenre_fix_leekeon_total6000.csv"
movie_df = pd.read_csv(label_path, encoding="ISO-8859-1")

In [ ]:
for movie in movie_df.values:
    #print(movie[3]) #link
    #print(movie[6]) #genre
    #print(movie[1]) #id
    
    # movie[0] ID
    # movie[7] Genre
    # movie[8] Image link 
    try:
        if(movie[7]=='Romance'):
            url = str(movie[3]) + '/plotsummary?ref_=tt_ov_pl'
            res = req.urlopen(url)
            soup = BeautifulSoup(res, "html.parser")   
            synop = soup.find('li', 'ipl-zebra-list__item')
            synop = synop.find('p')
            synop = str(synop).replace('</p>', '')
            synop = synop.replace('<p>', '')
    
            save_path = './dataset/poster_txt/new/Romance/' + str(movie[1]) + '.txt'
            file = open(save_path,'w')
            file.write(synop)
            file.flush()
    except:
        print(movie[1])

4188654
5566660


In [26]:
import pandas as pd
label_path = "./dataset/MovieGenre_genre500_final.csv"
movie_df = pd.read_csv(label_path, encoding="ISO-8859-1")

file = open('./dataset/poster_txt/Western/test.txt','w')
file.write("Hello World")
file.flush()

In [ ]:
from bs4 import BeautifulSoup
import urllib.request as req

####################################
# The Music Lyrics Database
####################################
site_url = "http://www.mldb.org/"

song_list = []
total_cnt = 0
cnt = 0

###############################################
# start function
###############################################
def get_all_songs(artist):
    global song_list
    global cnt
    
    song_list = []
    cnt = 0
    
    if len(artist) == 0:
        return
    
    artist = artist.strip().lower()
    
    get_search_list(artist)
    
    print('# ', artist, " : ", cnt)
        
    return

###############################################
# 1st page song list and other search page list
###############################################
def get_search_list(artist):
    url = site_url + "search?si=1&mm=2&ob=1&mq=" + artist.replace(' ', '+')
    
    # print('url : ' , url)
    res = req.urlopen(url)
    soup = BeautifulSoup(res, "html.parser")
    links = soup.find_all("a")
    b4txt = ""

    for a in links:
        href = a.attrs['href']
        text = a.string.lower()
        
        if b4txt.find(artist) >= 0 and href.startswith('song'):
            # print(text , " > ", href)
            get_song_lyrics(artist, text, href)
        elif href.startswith('search'):
            # print(text , " : ", href)
            get_song_list(artist, href)
        else:
            b4txt = text
            # print(text , " = ", href)

    return

##########################################
# after 1st page song list, from 2nd page
##########################################
def get_song_list(artist, param):
    url = site_url + param
    
    # print('url : ' , url)
    res = req.urlopen(url)
    soup = BeautifulSoup(res, "html.parser")
    links = soup.find_all("a")
    b4txt = ""
  
    for a in links:
        href = a.attrs['href']
        text = a.string.lower()
        
        if b4txt.find(artist) >= 0 and href.startswith('song'):
            # print(text , " > ", href)
            get_song_lyrics(artist, text, href)
        else:
            b4txt = text
            
    return

####################################
# get each song lyrics to write file
####################################
def get_song_lyrics(artist, title, param):
    
    global song_list
    global max_cnt, cnt, total_cnt
    
    if cnt >= 50:
        print('# max ', total_cnt, " ", cnt)
        return    
    
    pos = title.find('(')
    if (pos > 0): title = title[:pos].strip()
    
    if title in song_list:
        print('# already get : ', title)
        return
    
    song_list.append(title)
    
    url = site_url + param
    # print('# ', total_cnt, " ", cnt, " ", artist , ' : ', title, ' : ' , url)
        
    res = req.urlopen(url)
    soup = BeautifulSoup(res, "html.parser")
    
    p_str = str(soup.find('p'))
    p_str = p_str[(p_str.find('>')+1):]
    p_str = p_str.replace('<br/>', ' ')
    p_str = p_str.replace('</p>', '')
    
    # print(p_str)
    
    file_name = 'lyrics/' + artist + '_' +title + '.txt'
    print('# file : ', cnt, ' ', file_name)
    
    try:
        with open(file_name, 'w', encoding='utf8') as f:
            f.write(str(p_str))
            cnt = cnt+1
    except OSError as err:
        print("OS error: {0}".format(err))
    
    return